<div style="padding:20px;color:white;margin:0;font-size:250%;text-align:left;display:fill;border-radius:2px;background-color:#222222;overflow:hidden;font-weight:500">Интерактивный Dashboard</div>

При помощи открытого API - api.coincap.io собраны исторические данные о стоимости 100 различных криптовалют в долларах.

Используя библиотеку Dash разработан интерактивный dashboard на локальном сервере.
Два поля для фильтрации:
- Дата.
- Обозначение криптовалюты.



In [1]:
# Импорт необходимых библиотек

import requests
import pandas as pd
import numpy as np
from datetime import datetime as dt

import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input

<div style="padding:10px;color:white;margin:0;font-size:150%;text-align:left;display:fill;border-radius:2px;background-color:#079A82;overflow:hidden;font-weight:500">Сбор данных через API</div>

In [2]:
# Функция для получения доступного списка криптовалют

def crypto_values(url):
    list_crypto = []
    n = 0
    response = requests.get(url)
    data = response.json()
    while n != len(data['data']):
        list_crypto.append({data['data'][n]['id']: data['data'][n]['symbol']})
        n += 1
    return list_crypto

In [3]:
cryptos = crypto_values('https://api.coincap.io/v2/assets')

In [4]:
df_cols = ['name', 'symbol', 'priceUsd', 'date']
rows = []

In [5]:
#Функция для сбора информации о криптовалютах

def crypto_scraper(crypto_list):
    
    crypto_names = crypto_list
    
    while len(crypto_list) != 0 :
        
        for i in crypto_list:
        
            res = requests.get(f'http://api.coincap.io/v2/assets/{list(i.keys())[0]}/history?interval=d1')
    
            if res.status_code == 200:  #Некоторые данные периодически не доступны
                                        #Функция будет собирать пока не соберет полностью
                data = res.json()
                n = 0
        
                for num in range(0, len(data['data'])):
            
                    rows.append({'name': list(i.keys())[0],
                                 'symbol': i[list(i.keys())[0]],
                                 'date': data['data'][num]['date'],
                                 'priceUsd': data['data'][num]['priceUsd']})
                
                crypto_names.remove(i)
            
                
            else:
                print(f'Did not save {i}, fixing....')
    return print(f'All {len(rows)} rows saved!')
        
        
            

In [6]:
crypto_scraper(cryptos)

Did not save {'polygon': 'MATIC'}, fixing....
Did not save {'multi-collateral-dai': 'DAI'}, fixing....
Did not save {'ankr': 'ANKR'}, fixing....
Did not save {'iotex': 'IOTX'}, fixing....
Did not save {'avalanche': 'AVAX'}, fixing....
Did not save {'crypto-com-coin': 'CRO'}, fixing....
Did not save {'crypto-com-coin': 'CRO'}, fixing....
All 36200 rows saved!


In [7]:
df = pd.DataFrame(rows, columns = df_cols)
df.head()

,name,symbol,priceUsd,date
0,bitcoin,BTC,36585.7610417722410952,2022-01-25T00:00:00.000Z
1,bitcoin,BTC,37593.2862902953800372,2022-01-26T00:00:00.000Z
2,bitcoin,BTC,36443.7277806191196014,2022-01-27T00:00:00.000Z
3,bitcoin,BTC,37195.3337856870182247,2022-01-28T00:00:00.000Z
4,bitcoin,BTC,37936.3896273022133066,2022-01-29T00:00:00.000Z


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36200 entries, 0 to 36199
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      36200 non-null  object
 1   symbol    36200 non-null  object
 2   priceUsd  36200 non-null  object
 3   date      36200 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [10]:
df.head()

,name,symbol,priceUsd,date
0,bitcoin,BTC,36585.761042,2022-01-25T00:00:00.000Z
1,bitcoin,BTC,37593.286290,2022-01-26T00:00:00.000Z
2,bitcoin,BTC,36443.727781,2022-01-27T00:00:00.000Z
3,bitcoin,BTC,37195.333786,2022-01-28T00:00:00.000Z
4,bitcoin,BTC,37936.389627,2022-01-29T00:00:00.000Z


### Данные собраны! 

In [11]:
df.to_csv('crypto_new.csv')

<div style="padding:10px;color:white;margin:0;font-size:150%;text-align:left;display:fill;border-radius:2px;background-color:#079A82;overflow:hidden;font-weight:500">Построение Dashboard</div>

In [ ]:
#Подготовка данных

data = df
data['Date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['priceUsd'] = pd.to_numeric(df['priceUsd'])
data.sort_values('Date', inplace=True)
                              

# Стиль визуализации                               
external_stylesheets = [
    {
        'href': 'https://fonts.googleapis.com/css2?'
        'family=Lato:wght@400;700&display=swap',
        'rel': 'stylesheet',
    },
]
                              

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Cryptos!'
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children='Cryptocurrency price by date', className='header-title'
                ),
                html.P(
                    children='CRYPTOCURRENCY RATE DYNAMICS',
                    className='header-description',
                ),
            ],
            className='header',
        ),
        
        # Фильтр по криптовалюте
        
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children='Symbol', className='menu-title'),
                        dcc.Dropdown(
                            id='symbol-filter',
                            options=[
                                {'label': symbol, 'value': symbol}
                                for symbol in np.sort(data.symbol.unique())
                            ],
                            value='BTC',
                            clearable=False,
                            className='dropdown',
                        ),
                    ]
                ),
                
                # Фильтр по дате
                html.Div(
                    children=[
                        html.Div(
                            children='Date Range',
                            className='menu-title'
                            ),
                        dcc.DatePickerRange(
                            id='date-range',
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className='menu',
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id='bar-chart', config={'displayModeBar': False},
                    ),
                    className='card',
                ),
                html.Div(
                    children=dcc.Graph(
                        id='line-chart', config={'displayModeBar': False},
                    ),
                    className='card',
                ),
            ],
            className='wrapper',
        ),
    ]
)
@app.callback(
    [Output('bar-chart', 'figure'), Output('line-chart', 'figure')],
    [
        Input('symbol-filter', 'value'),
        Input('date-range', 'start_date'),
        Input('date-range', 'end_date'),
    ],
)
                              
# Функция фильтрующая 
def update_charts(symbol, start_date, end_date):
    mask = (
        (data.symbol == symbol)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    
    bar_chart_figure = {
        'data': [{'x': filtered_data['Date'],
                'y': filtered_data['priceUsd'],
                'type': 'bar',
                'hovertemplate': '$%{y:.2f}',
            },
        ],
        'layout': {
            'title': {
                'text': 'Rate bar chart',
                'x': 0.05,
                'xanchor': 'left',
            },
            'xaxis': {'fixedrange': True},
            'yaxis': {'tickprefix': '$', 'fixedrange': True},
            'colorway': ['#17B897'],
        },
    }
    line_chart_figure = {
        'data': [
            {
                'x': filtered_data['Date'],
                'y': filtered_data['priceUsd'],
                'type': 'lines',
            },
        ],
        'layout': {
            'title': {'text': 'Rate Line chart', 'x': 0.05, 'xanchor': 'left'},
            'xaxis': {'fixedrange': True},
            'yaxis': {'fixedrange': True},
            'colorway': ['#E12D39'],
        },
    }
    return bar_chart_figure, line_chart_figure
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:03:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2023 02:04:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:04:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2023 02:04:05] "GET /_dash-dependencies HTTP/1.1" 200 -
12